<a href="https://colab.research.google.com/github/surya-prakash-susarla/MultiProcessTestCodes/blob/master/CommonProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [379]:
from google.colab import drive
drive.mount('/content/drive')

k_training_data_path = '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TrainingData/'
k_testing_data_path = '/content/drive/MyDrive/ECE542_sp2022_Project_TerrainRecognition/TestData/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [380]:
import os

training_data_files = os.listdir(k_training_data_path)
testing_data_files = os.listdir(k_testing_data_path)

print('Total training data files found : ', len(training_data_files))
print('Total test data files found : ', len(testing_data_files))

Total training data files found :  116
Total test data files found :  12


In [381]:
import glob

# Training data only.
x_value_paths = glob.glob( k_training_data_path + '*_x.csv')
x_time_paths = glob.glob( k_training_data_path + '*__x_time.csv')

y_value_paths = glob.glob( k_training_data_path + '*_y.csv')
y_time_paths = glob.glob ( k_training_data_path + '*__y_time.csv')

x_value_paths.sort()
x_time_paths.sort()
y_value_paths.sort()
y_time_paths.sort()

print('x reading files : ', len(x_value_paths))
print('x time files : ', len(x_time_paths))

print('y value files : ', len(y_value_paths))
print('y time files : ', len(y_time_paths))

x reading files :  29
x time files :  29
y value files :  29
y time files :  29


In [382]:
import pandas as pd
import numpy as np

def merge_csv_files(filepaths):
  result = pd.DataFrame()
  for path in filepaths:
    incoming_data = pd.read_csv(path)
    if result.size == 0:
      result = pd.DataFrame(np.array(incoming_data.values))
    else:
      result = pd.DataFrame(np.concatenate((result.values, incoming_data.values), axis=0))
  return result

# _x files
data_x = merge_csv_files(x_value_paths)
print("Entries in data_x - ", len(data_x))

# _x_time files
data_x_time = merge_csv_files(x_time_paths)
print("Entries in data_x_time - ", len(data_x_time))

# _y files
data_y = merge_csv_files(y_value_paths)
print("Entries in data_y - ", len(data_y))

# _x_time files
data_y_time = merge_csv_files(y_time_paths)
print("Entries in data_y_time - ", len(data_y_time))

Entries in data_x -  1341617
Entries in data_x_time -  1341617
Entries in data_y -  335384
Entries in data_y_time -  335384


In [383]:
# TEST ONLY ( REDUCE DATA FOR QUICK VALIDATION - REMOVE FOR ACTUAL TESTS )

rs = .5

data_x = data_x[:(int)(len(data_x)*rs)]
data_x_time = data_x_time[:(int)(len(data_x_time)*rs)]
data_y = data_y[:(int)(len(data_y)*rs)]
data_y_time = data_y_time[:(int)(len(data_y_time)*rs)]


print('reduced data size : ')
print(len(data_x))
print(len(data_x_time))
print(len(data_y))
print(len(data_y_time))


reduced data size : 
670808
670808
167692
167692


In [384]:
import numpy as np

# Pick every 4th element in the x values
data_x_pruned = data_x.iloc[::4, :]

print('original size : ', len(data_x))
print("expected size : ", len(data_x)/4)
print('dumb prune size : ', len(data_x_pruned))
print('y size : ', len(data_y))
print('gap : ', len(data_x_pruned) - len(data_y))

data_x_pruned = data_x_pruned[:-(len(data_x_pruned) - len(data_y))]

print('gap with updated size : ', np.abs(len(data_x_pruned) - len(data_y)))

original size :  670808
expected size :  167702.0
dumb prune size :  167702
y size :  167692
gap :  10
gap with updated size :  0


In [385]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [386]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data_x_pruned, data_y, test_size=0.3, random_state=42)

print("x train shape : ", x_train.shape)
print("y train shape : ", y_train.shape)
print("x test shape : ", x_test.shape)
print("y test shape : ", y_test.shape)

x train shape :  (117384, 6)
y train shape :  (117384, 1)
x test shape :  (50308, 6)
y test shape :  (50308, 1)


In [387]:
# model = keras.Sequential()
# model.add(layers.Dense(7, input_dim=6))
# model.add(layers.Dense(1))
# model.build()
# model.summary()

# model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# temp = np.array(x_train)

# model.fit(x_train, y_train, epochs=1, batch_size=10)
# _, acc = model.evaluate(x_test, y_test)
# print("Model accuracy : ", acc)

In [388]:
# Data processing for LSTM windows


print(data_x.shape)
print(data_x_time.shape)

# append timestamp as 0th col
# full_x = data_x
# full_x.insert(0, 'time', data_x_time)

print(data_y.shape)
print(data_y_time.shape)

# append timestamp as 0th col
# full_y = data_y
# full_y.insert(0, 'time', data_y_time)

# print("append shapes")
# print(full_x.shape)
# print(full_y.shape)

# print("sample")
# print(full_x.head(n=20))
# print(full_y.head())


# for i in range(1, len(data_y_time)):
#   if (data_y_time.iloc[i, 0] - data_y_time.iloc[i-1, 0]) > 10:
#     print("diff > 10 identified for y1 : {}, y2 : {}".format(data_y_time.iloc[i, 0], data_y_time.iloc[i-1, 0]))

print('x sample : ')
print(data_x.head())

l_x = data_x

l_y = []
# duplicate y  values
i = 0
j = 0
while True:
  if i == len(l_x) or j == len(data_y):
    break
  if data_y_time.iloc[j, 0] >= l_x.iloc[i, 0]:
    l_y.append(data_y.loc[j])
    i+=1
  else:
    j+=1

while i != len(l_x):
  l_y.append(l_y[-1])
  i += 1

l_x_train, l_x_test, l_y_train, l_y_test = train_test_split(l_x, l_y, test_size=0.3, random_state=42)

(670808, 6)
(670808, 1)
(167692, 1)
(167692, 1)
x sample : 
          0         1         2         3         4         5
0  4.186920  8.344455  2.908057  0.005771 -0.004480 -0.003345
1  4.544637  8.408659  2.890000  0.007967  0.022412  0.001159
2  4.849308  8.411614  2.900692  0.027778 -0.010670 -0.014223
3  4.509190  8.118649  2.847298  0.021577 -0.045498 -0.021111
4  4.226515  8.273807  2.851742  0.012534  0.000445 -0.016830


In [389]:
l_y_train = np.array(l_y_train)
l_y_test = np.array(l_y_test)

print(l_x_train.shape)
print(l_y_train.shape)
print(l_x_test.shape)
print(l_y_test.shape)

(469565, 6)
(469565, 1)
(201243, 6)
(201243, 1)


In [390]:
l_x_train = pd.DataFrame(l_x_train)
l_x_test = pd.DataFrame(l_x_test)

print(l_x_train.shape)
print(l_x_test.shape)

# LSTM only modifications
l_x_train = np.reshape(l_x_train.values, (l_x_train.shape[0], 1, l_x_train.shape[1]))
l_x_test = np.reshape(l_x_test.values, (l_x_test.shape[0], 1, l_x_test.shape[1]))

# temp_x_train = []
# temp_x_test = []

# print('shape : ', l_x_train.values.shape)

# for i in range(0, len(l_x_train), 4):
#   temp_x_train = np.append(temp_x_train, l_x_train.values[i:i+4])

# for i in range(0, len(l_x_test), 4):
#   temp_x_test = np.append(temp_x_test, l_x_test.values[i:i+4])

# print(type(temp_x_train[0]))
# print(temp_x_train[0])
# print(len(temp_x_train))

# # temp_x_train = np.array(temp_x_train)
# # temp_x_test = np.array(temp_x_test)

# l_x_train = temp_x_train
# l_x_test = temp_x_test

# print(l_x_train.shape)
# print(l_x_test.shape)

# print("updated shape : ", l_x_train.shape)


lstm_model = keras.Sequential()
lstm_model.add(layers.LSTM(6, input_dim=6))
lstm_model.add(layers.Dense(1))
lstm_model.compile(loss='mean_squared_error', optimizer='adam')
lstm_model.fit(l_x_train, l_y_train, epochs=1, batch_size=2)

acc = lstm_model.evaluate(l_x_test, l_y_test)
print("Model accuracy : ", acc)

(469565, 6)
(201243, 6)
6289/6289 [==============================] - 12s 2ms/step - loss: 4.5549e-11
Model accuracy :  4.554861349204131e-11
